In [ ]:

!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 32.1 M

In [ ]:

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cu118 and Flower 1.5.0


In [ ]:
CLASSES = (
    "EOSINOPHIL",
    "LYMPHOCYTE",
    "MONOCYTE",
    "NEUTROPHIL",
)


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Changed input channels from 1 to 3
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)  # Adjusted input size based on pooling and conv layers
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)


In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=1,
    min_available_clients=3,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)


INFO flwr 2023-09-03 03:33:23,201 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-09-03 03:33:26,759	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-03 03:33:29,721 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 3886889779.0, 'memory': 7773779559.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 3886889779.0, 'memory': 7773779559.0}
INFO flwr 2023-09-03 03:33:29,729 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-03 03:33:29,743 | app.py:224 | Flower VCE: Resources for each Virtual Clie

(DefaultActor pid=2001) [Client 2] fit, config: {}
(DefaultActor pid=1999) Epoch 1: train loss 0.041133396327495575, accuracy 0.39707112970711295
(DefaultActor pid=1999) [Client 1] fit, config: {}
(DefaultActor pid=1999) [Client 0] fit, config: {}


DEBUG flwr 2023-09-03 03:55:23,853 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures


(DefaultActor pid=1999) Epoch 1: train loss 0.0409831777215004, accuracy 0.399581589958159 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


WARNING flwr 2023-09-03 03:55:24,140 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-09-03 03:55:24,145 | server.py:173 | evaluate_round 1: strategy sampled 1 clients (out of 3)
DEBUG:flwr:evaluate_round 1: strategy sampled 1 clients (out of 3)


...

(DefaultActor pid=1999) [Client 2] fit, config: {}
(DefaultActor pid=1999) Epoch 1: train loss 0.0357535183429718, accuracy 0.48493723849372383
(DefaultActor pid=2001) [Client 0] fit, config: {}
(DefaultActor pid=1999) [Client 1] fit, config: {}
(DefaultActor pid=2001) Epoch 1: train loss 0.034476786851882935, accuracy 0.5133891213389121


DEBUG flwr 2023-09-03 04:03:30,299 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-09-03 04:03:30,426 | server.py:173 | evaluate_round 3: strategy sampled 1 clients (out of 3)
DEBUG:flwr:evaluate_round 3: strategy sampled 1 clients (out of 3)


(DefaultActor pid=1999) Epoch 1: train loss 0.03506774082779884, accuracy 0.4920502092050209
(DefaultActor pid=1999) [Client 0] evaluate, config: {}


DEBUG flwr 2023-09-03 04:04:33,186 | server.py:187 | evaluate_round 3 received 1 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 1 results and 0 failures
INFO flwr 2023-09-03 04:04:33,191 | server.py:153 | FL finished in 1863.3823381760003
INFO:flwr:FL finished in 1863.3823381760003
INFO flwr 2023-09-03 04:04:33,196 | app.py:225 | app_fit: losses_distributed [(1, 0.04197251526814587), (2, 0.039351245592225276), (3, 0.037191133229237686)]
INFO:flwr:app_fit: losses_distributed [(1, 0.04197251526814587), (2, 0.039351245592225276), (3, 0.037191133229237686)]
INFO flwr 2023-09-03 04:04:33,199 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-03 04:04:33,201 | app.py:227 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-09-03 04:04:33,203 | app.py:228 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-09-03 04:04:33,205 | app.py:229 | app

History (loss, distributed):
	round 1: 0.04197251526814587
	round 2: 0.039351245592225276
	round 3: 0.037191133229237686

In [ ]:

# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-09-03 04:08:46,245 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-09-03 04:08:51,137	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-03 04:08:53,838 | app.py:210 | Flower VCE: Ray initialized with resources: {'memory': 7776323175.0, 'object_store_memory': 3888161587.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7776323175.0, 'object_store_memory': 3888161587.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-09-03 04:08:53,846 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-03 04:08:53,849 | app.py:224 | Flower VCE: Resources for each Virtual Clie

(DefaultActor pid=10939) [Client 1] fit, config: {}
(DefaultActor pid=10938) Epoch 1: train loss 0.04304547607898712, accuracy 0.35648535564853556
(DefaultActor pid=10938) [Client 2] fit, config: {}
(DefaultActor pid=10938) [Client 0] fit, config: {}


DEBUG flwr 2023-09-03 04:12:59,058 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-09-03 04:12:59,196 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-09-03 04:12:59,292 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 3)


(DefaultActor pid=10938) Epoch 1: train loss 0.04054262861609459, accuracy 0.4171548117154812 [repeated 2x across cluster]
(DefaultActor pid=10938) [Client 2] evaluate, config: {}


DEBUG flwr 2023-09-03 04:14:05,412 | server.py:187 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-09-03 04:14:05,415 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-09-03 04:14:05,419 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 3)


...

(DefaultActor pid=10939) Epoch 1: train loss 2.374307870864868, accuracy 0.2594142259414226


DEBUG flwr 2023-09-03 04:21:08,185 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 3)


(DefaultActor pid=10939) [Client 0] evaluate, config: {}
(DefaultActor pid=10939) [Client 2] evaluate, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-09-03 04:21:18,051 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-09-03 04:21:18,056 | server.py:153 | FL finished in 744.1448925119998
INFO:flwr:FL finished in 744.1448925119998
INFO flwr 2023-09-03 04:21:18,058 | app.py:225 | app_fit: losses_distributed [(1, 112.35262658461086), (2, 33.854831681161556), (3, 0.28944834943087594)]
INFO:flwr:app_fit: losses_distributed [(1, 112.35262658461086), (2, 33.854831681161556), (3, 0.28944834943087594)]
INFO flwr 2023-09-03 04:21:18,060 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-03 04:21:18,062 | app.py:227 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-09-03 04:21:18,064 | app.py:228 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-09-03 04:21:18,066 | app.py:229 | app_fit: metr

History (loss, distributed):
	round 1: 112.35262658461086
	round 2: 33.854831681161556
	round 3: 0.28944834943087594

In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)


INFO flwr 2023-09-03 04:22:25,360 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-09-03 04:22:31,726	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-09-03 04:22:35,085 | app.py:210 | Flower VCE: Ray initialized with resources: {'memory': 7805728359.0, 'object_store_memory': 3902864179.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7805728359.0, 'object_store_memory': 3902864179.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO flwr 2023-09-03 04:22:35,101 | app.py:218 | No `client_resources` specified. Using minimal resources for clients.
INFO:flwr:No `client_resources` specified. Using minimal resources for clients.
INFO flwr 2023-09-03 04:22:35,134 | app.py:224 | Flower VCE: Resources for each Virtual Clie

Server-side evaluation loss 0.07887210666008715 / accuracy 0.0


(pid=14540) 2023-09-03 04:22:46.427613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=14540) [Client 0] fit, config: {}
(DefaultActor pid=14539) Epoch 1: train loss 0.04214601218700409, accuracy 0.3815899581589958
(DefaultActor pid=14539) [Client 2] fit, config: {}
(DefaultActor pid=14539) [Client 1] fit, config: {}


DEBUG flwr 2023-09-03 04:26:37,246 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-09-03 04:26:37,350 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


...

(DefaultActor pid=14540) Epoch 1: train loss 0.029695365577936172, accuracy 0.6083682008368201 [repeated 2x across cluster]


INFO flwr 2023-09-03 04:33:57,002 | server.py:125 | fit progress: (3, 0.022709498990256833, {'accuracy': 0.7094339622641509}, 674.2373173249998)
INFO:flwr:fit progress: (3, 0.022709498990256833, {'accuracy': 0.7094339622641509}, 674.2373173249998)
DEBUG flwr 2023-09-03 04:33:57,006 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 3)


Server-side evaluation loss 0.022709498990256833 / accuracy 0.7094339622641509
(DefaultActor pid=14540) [Client 0] evaluate, config: {}
(DefaultActor pid=14540) [Client 1] evaluate, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-09-03 04:34:08,408 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-09-03 04:34:08,411 | server.py:153 | FL finished in 685.646486787
INFO:flwr:FL finished in 685.646486787
INFO flwr 2023-09-03 04:34:08,413 | app.py:225 | app_fit: losses_distributed [(1, 0.041441827600107256), (2, 0.036602421526638965), (3, 0.023509229654036227)]
INFO:flwr:app_fit: losses_distributed [(1, 0.041441827600107256), (2, 0.036602421526638965), (3, 0.023509229654036227)]
INFO flwr 2023-09-03 04:34:08,416 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-03 04:34:08,418 | app.py:227 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-09-03 04:34:08,420 | app.py:228 | app_fit: losses_centralized [(0, 0.07887210666008715), (1, 0.040843392318149783), (2, 0.03600107993719713), (3, 0.022709498990256833

History (loss, distributed):
	round 1: 0.041441827600107256
	round 2: 0.036602421526638965
	round 3: 0.023509229654036227
History (loss, centralized):
	round 0: 0.07887210666008715
	round 1: 0.040843392318149783
	round 2: 0.03600107993719713
	round 3: 0.022709498990256833
History (metrics, centralized):
{'accuracy': [(0, 0.0), (1, 0.47547169811320755), (2, 0.47924528301886793), (3, 0.7094339622641509)]}

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config